1. folder是用絕對路徑寫的，要記得改
2. 然後folder的結構是有mass train和 mass test
3. 實作是將兩個folder裡面都各自創benign和malignant的folder，然後把對應的crop image放到裡面

In [1]:
import pydicom
import cv2
import numpy as np
import pandas as pd
import csv
import os

if you are running train, run this code

In [27]:
clinical_train_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328","mass_case_description_train_set.csv")
clinical_test_path = os.path.join("/Volumes/Extreme/roi_data/manifest-OlX3eVq41551709337535597984","mass_case_description_test_set.csv")
metadata_train_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328","metadata.csv")
metadata_test_path = os.path.join("/Volumes/Extreme/roi_data/manifest-OlX3eVq41551709337535597984","metadata.csv")
train_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328")
test_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-OlX3eVq41551709337535597984")
malignant_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328","malignant")
benign_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328","benign")
clinical_train = pd.read_csv(clinical_train_path, dtype=str)
clinical_test = pd.read_csv(clinical_test_path, dtype=str)
metadata_train = pd.read_csv(metadata_train_path, dtype=str)
metadata_test = pd.read_csv(metadata_test_path, dtype=str)
print(np.sum(clinical_train["pathology"] == "MALIGNANT"))
print(np.sum(clinical_train["pathology"] == "BENIGN"))
print(np.sum(clinical_test["pathology"] == "MALIGNANT"))
print(np.sum(clinical_test["pathology"] == "BENIGN"))
label = []
subjectID = []
for i,data in enumerate(clinical_train.iloc):
    val = "malignant" if data["pathology"] == "MALIGNANT" else "benign"
    subjectID.append(data["cropped image file path"].split("/")[0])
    label.append(val)
# print(len(metadata_train), len(label))
for i,data in enumerate(metadata_train.iloc):
    dcm_pth = train_image_path + data["File Location"][1:]
    if dcm_pth.split("/")[6][0] == '.':
        continue
    ID = data["Subject ID"]
    # print(dcm_pth)
    for j,filename in enumerate(os.listdir(dcm_pth)): 
        if filename[0] == '.':
            continue
        index = subjectID.index(ID)
        if label[i] == "malignant":
            png_pth = malignant_image_path + "/"
        else:
            png_pth = benign_image_path + "/"
        png_pth = png_pth + ID + "-" + filename[:-4] + ".png"
        dicom_image = pydicom.dcmread(dcm_pth + "/" + filename)
        if dicom_image[("0008", "103e")].value == "cropped images":
            cv2.imwrite(png_pth, dicom_image.pixel_array)

637
577
147
194


IndexError: list index out of range

if you are running test, run this code

In [31]:
label = []
subjectID = []
malignant_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-OlX3eVq41551709337535597984","malignant")
benign_image_path = os.path.join("/Volumes/Extreme/roi_data/manifest-OlX3eVq41551709337535597984","benign")
for i,data in enumerate(clinical_test.iloc):
    val = "malignant" if data["pathology"] == "MALIGNANT" else "benign"
    subjectID.append(data["cropped image file path"].split("/")[0])
    label.append(val)
for i,data in enumerate(metadata_test.iloc):
    dcm_pth = test_image_path + data["File Location"][1:]
    if dcm_pth.split("/")[6][0] == '.':
        continue
    ID = data["Subject ID"]
    for j,filename in enumerate(os.listdir(dcm_pth)): 
        if filename[0] == '.':
            continue
        index = subjectID.index(ID)
        if label[i] == "malignant":
            png_pth = malignant_image_path + "/"
        else:
            png_pth = benign_image_path + "/"
        png_pth = png_pth + ID + "-" + filename[:-4] + ".png"
        dicom_image = pydicom.dcmread(dcm_pth + "/" + filename)
        if dicom_image[("0008", "103e")].value == "cropped images":
            cv2.imwrite(png_pth, dicom_image.pixel_array)

IndexError: list index out of range

In [25]:
path = '/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328/malignant'
new_path_p = '/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328/malignant_preprocess'
def preprocess(image):
    img = cv2.equalizeHist(image)
    gaussian_3 = cv2.GaussianBlur(img, (0, 0), 2.0)
    unsharp_image = cv2.addWeighted(img, 2.0, gaussian_3, -1.0, 0)
    median_image = cv2.medianBlur(unsharp_image, 3)
    resized = cv2.resize(median_image, (224,224))
    color_image = cv2.merge([resized] * 3)
    return color_image

train_path = os.path.join("/Volumes/Extreme/roi_data/manifest-LyDgOQGl3853937313152078328")
train_image_path = []
train_label = []
for i,filename in enumerate(os.listdir(train_path+"/benign")):
    train_image_path.append((train_path+"/benign/"+filename,"benign"))
for i,filename in enumerate(os.listdir(train_path+"/malignant")):
    train_image_path.append((train_path+"/malignant/"+filename,"malignant"))
for path,l in train_image_path:
    img = cv2.imread(path,0)
    hist = cv2.equalizeHist(img)
    gaussian_3 = cv2.GaussianBlur(hist, (0, 0), 2.0)
    unsharp_image = cv2.addWeighted(hist, 2.0, gaussian_3, -1.0, 0)
    median_image = cv2.medianBlur(unsharp_image, 3)
    img_bil = cv2.bilateralFilter(median_image, 5, 120, 120)
    cv2.imwrite(path, img_bil)